<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=250, height=250>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

# Задание 3

## Классификация текстов

В этом задании вам предстоит попробовать несколько методов, используемых в задаче классификации, а также понять насколько хорошо модель понимает смысл слов и какие слова в примере влияют на результат.

In [1]:
import pandas as pd
import numpy as np
import torch

from torchtext.legacy import datasets

from torchtext.legacy.data import Field, LabelField
from torchtext.legacy.data import BucketIterator

from torchtext.vocab import Vectors, GloVe

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
from tqdm.autonotebook import tqdm

In [2]:
import warnings
warnings.filterwarnings("ignore")

В этом задании мы будем использовать библиотеку torchtext. Она довольна проста в использовании и поможет нам сконцентрироваться на задаче, а не на написании Dataloader-а.

In [9]:
TEXT = Field(sequential=True, lower=True, include_lengths=True)  # Поле текста # sequential=True т.к. входные данные - предложения и их нужно токенизировать
LABEL = LabelField(dtype=torch.float)  # Поле метки

In [3]:
SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Датасет на котором мы будем проводить эксперементы это комментарии к фильмам из сайта IMDB.

In [10]:
train_Dataset, test_Dataset = datasets.IMDB.splits(TEXT, LABEL)  # загрузим датасет
train_Dataset, valid_Dataset = train_Dataset.split(random_state=random.seed(SEED))  # разобьем на части

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:05<00:00, 15.2MB/s]


In [30]:
type(train_Dataset)

torchtext.legacy.data.dataset.Dataset

In [31]:
len(train_Dataset), len(valid_Dataset), len(test_Dataset)

(17500, 7500, 25000)

In [32]:
for example in train_Dataset:
    print(type(example))
    print(dir(example))
    break

<class 'torchtext.legacy.data.example.Example'>
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'fromCSV', 'fromJSON', 'fromdict', 'fromlist', 'fromtree', 'label', 'text']


In [12]:
TEXT.build_vocab(train_Dataset)
LABEL.build_vocab(train_Dataset)

In [34]:
LABEL.vocab.freqs

Counter({'neg': 8810, 'pos': 8690})

In [35]:
TEXT.vocab.freqs

Counter({'this': 48843,
         'movie': 21880,
         "doesn't": 3166,
         'even': 8450,
         'deserve': 187,
         'a': 112448,
         'one.': 817,
         'was': 33019,
         'an': 14839,
         'utter': 170,
         'waste': 985,
         'of': 101203,
         'time.': 1380,
         'it': 46019,
         'film': 19578,
         'and': 111274,
         'money.': 213,
         'not': 20127,
         'offensive': 98,
         'but': 27903,
         'everything': 1377,
         'provocative': 37,
         'disgusting.': 24,
         'my': 8170,
         'spoiler': 112,
         'is': 73124,
         'one': 15721,
         'that': 46644,
         'i': 49537,
         'think': 4751,
         'should': 3364,
         'be': 17979,
         'read': 1247,
         'by': 15460,
         'everyone.': 91,
         'there': 9313,
         'full': 1081,
         'frontal': 32,
         'nudity': 263,
         'disgusting': 104,
         'language.': 37,
         'only': 

In [42]:
len(TEXT.vocab.stoi)

202779

In [ ]:
list(TEXT.vocab.stoi.items())[-10:]

[('“him”', 201373),
 ('“it’s', 201374),
 ('“jean', 201375),
 ('“little', 201376),
 ('“mad', 201377),
 ('“mr.', 201378),
 ('“playboy”', 201379),
 ('“sanatorium”', 201380),
 ('“x”,', 201381),
 ('₤100', 201382)]

In [ ]:
TEXT.vocab.stoi["₤100"]

201382

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torchtext_train_dataloader, torchtext_valid_dataloader, torchtext_test_dataloader = BucketIterator.splits(
    (train_Dataset, valid_Dataset, test_Dataset), 
    batch_size=64,
    device = device,
    sort_key=lambda x: len(x.text),
    sort=False,
    shuffle=True,
    sort_within_batch=True
    )

In [ ]:
torchtext_train_dataloader.create_batches()
torchtext_valid_dataloader.create_batches()
torchtext_test_dataloader.create_batches()

In [ ]:
dir(torchtext_train_dataloader)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_iterations_this_epoch',
 '_random_state_this_epoch',
 '_restored_from_state',
 'batch_size',
 'batch_size_fn',
 'batches',
 'create_batches',
 'data',
 'dataset',
 'device',
 'epoch',
 'init_epoch',
 'iterations',
 'load_state_dict',
 'random_shuffler',
 'repeat',
 'shuffle',
 'sort',
 'sort_key',
 'sort_within_batch',
 'splits',
 'state_dict',
 'train']

In [ ]:
[len(text.text) for text in torchtext_train_dataloader.data()] # распределение длин предложений 

[364,
 377,
 198,
 119,
 132,
 119,
 138,
 110,
 170,
 147,
 179,
 182,
 465,
 282,
 216,
 540,
 272,
 240,
 184,
 981,
 462,
 183,
 463,
 168,
 103,
 147,
 182,
 247,
 171,
 82,
 126,
 765,
 134,
 267,
 62,
 371,
 439,
 90,
 54,
 139,
 559,
 103,
 153,
 510,
 179,
 775,
 199,
 253,
 53,
 79,
 283,
 104,
 406,
 211,
 528,
 125,
 116,
 177,
 114,
 216,
 212,
 141,
 455,
 278,
 159,
 346,
 202,
 605,
 173,
 173,
 103,
 333,
 158,
 100,
 84,
 182,
 228,
 309,
 104,
 522,
 117,
 101,
 163,
 78,
 89,
 102,
 99,
 147,
 301,
 121,
 156,
 131,
 479,
 62,
 200,
 144,
 101,
 327,
 238,
 222,
 216,
 156,
 49,
 71,
 121,
 127,
 115,
 298,
 110,
 477,
 145,
 114,
 244,
 138,
 232,
 239,
 124,
 173,
 396,
 109,
 110,
 75,
 142,
 631,
 150,
 133,
 58,
 127,
 150,
 151,
 182,
 139,
 296,
 156,
 335,
 150,
 188,
 193,
 272,
 132,
 156,
 60,
 352,
 137,
 401,
 67,
 226,
 129,
 223,
 587,
 545,
 558,
 456,
 202,
 226,
 267,
 305,
 122,
 451,
 102,
 174,
 239,
 181,
 506,
 137,
 124,
 275,
 56,
 172,
 170

In [ ]:
len(torchtext_train_dataloader.data()) # 17500 примеров в torchtext_train_dataloader

17500

In [ ]:
len(torchtext_valid_dataloader.data()) # 7500 примеров для валидации

7500

In [ ]:
len(torchtext_test_dataloader.data()) # 25000 примеров для теста

25000

In [ ]:
for batch_no, batch in enumerate(torchtext_train_dataloader):
    text, batch_len = batch.text # text.size() -> seq_len, batch_size
    print(text, batch_len, sep="\n")
    print(batch.label)
    break

tensor([[     9,     49,     10,  ...,   7828,      3,  94314],
        [    85,      9,     20,  ...,     10,    764, 141347],
        [    98,     82,     14,  ...,      7,   2525,     13],
        ...,
        [    24,    108, 198439,  ...,      1,      1,      1],
        [    15,    103,     13,  ...,      1,      1,      1],
        [ 14562, 179732,    728,  ...,      1,      1,      1]],
       device='cuda:0')
tensor([297, 297, 297, 297, 297, 297, 296, 296, 296, 296, 296, 296, 295, 295,
        295, 295, 295, 295, 294, 294, 294, 294, 294, 294, 293, 293, 293, 293,
        293, 293, 293, 293, 293, 292, 292, 292, 292, 292, 292, 291, 291, 291,
        291, 290, 290, 290, 290, 290, 290, 290, 289, 289, 289, 289, 289, 289,
        289, 289, 288, 288, 288, 288, 288, 288], device='cuda:0')
tensor([1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0.,
        1., 1., 0., 0., 0., 1., 1., 1., 

In [ ]:
text.size(), batch_len.size(), batch.label.size()

(torch.Size([297, 64]), torch.Size([64]), torch.Size([64]))

## RNN

Для начала попробуем использовать рекурентные нейронные сети. На семинаре вы познакомились с GRU, вы можете также попробовать LSTM. Можно использовать для классификации как hidden_state, так и output последнего токена.

In [ ]:
# Для инициализации self.rnn(см. ниже) очевидно нужно создать модель рекуррентной нейронной сети:
# Очевидно, это может быть:
# 1) RNN
# 2) GRU
# 3) LSTM

# Конечно же можно воспользоваться определёнными в семинаре рекуррентными блоками, но воспользуемся
# реализованными уже в PyTorch

In [4]:
from torch.nn import RNN, GRU, LSTM

In [69]:
class RNNBaseline(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        # YOUR CODE GOES HERE
        self.lstm = LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout) # TO DO
        # YOUR CODE GOES HERE
        self.fc = nn.Linear(2 * hidden_dim, output_dim)
        
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb_dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths).to(device)
        
        # cell arg for LSTM, remove for GRU
        packed_output, (hidden, cell) = self.lstm(packed_embedded)

        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)  

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)  # YOUR CODE GOES HERE
                
        #hidden = [batch size, hid dim * num directions] or [batch_size, hid dim * num directions]
        #print(hidden.size()) # (batch_size, 2 * hidden_size)
        return self.fc(hidden) # (batch_size, output_dim)

Поиграйтесь с гиперпараметрами

In [ ]:
vocab_size = len(TEXT.vocab) # размер словаря(кол-во слов в словаре)
emb_dim = 100 # размерность embeddings
hidden_dim = 256 # размерность скрытого состояния 
output_dim = 2 # кол-во выходных слоёв после линейного слоя
n_layers = 2 # кол - во рекуррентных ячеек
bidirectional = True
dropout = 0.2
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
patience=7

In [ ]:
model = RNNBaseline(
    vocab_size=vocab_size,
    embedding_dim=emb_dim,
    hidden_dim=hidden_dim,
    output_dim=output_dim,
    n_layers=n_layers,
    bidirectional=bidirectional,
    dropout=dropout,
    pad_idx=PAD_IDX
)

In [ ]:
model = model.to(device)

In [ ]:
opt = torch.optim.Adam(model.parameters())
loss_func = nn.BCEWithLogitsLoss()

Обучите сетку! Используйте любые вам удобные инструменты, Catalyst, PyTorch Lightning или свои велосипеды.

In [5]:
from sklearn.metrics import f1_score 

In [ ]:
import numpy as np

min_loss = np.inf

cur_patience = 0
max_epochs = 20

for epoch in range(1, max_epochs + 1):

    train_loss = 0.0
    train_f1_score = 0.0

    model.train()

    pbar = tqdm(enumerate(torchtext_train_dataloader), total=len(torchtext_train_dataloader), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    
    for it, batch in pbar: 

        #YOUR CODE GOES HERE
        text, text_lengths = batch.text
        y_true = batch.label
        
        opt.zero_grad()
        
        y_pred = model(text, text_lengths.cpu())
        #print(predictions.size())
        loss = loss_func(y_pred[:, 1], y_true)
        loss.backward()
        opt.step()

        train_loss += loss.detach().cpu().item()
        train_f1_score += f1_score(y_true.cpu().numpy(), torch.argmax(y_pred, dim=1).cpu().numpy())
    train_loss /= len(torchtext_train_dataloader)
    train_f1_score /= len(torchtext_train_dataloader)
    
    val_loss = 0.0
    val_f1_score = 0.0

    model.eval()

    pbar = tqdm(enumerate(torchtext_valid_dataloader), total=len(torchtext_valid_dataloader), leave=False)
    pbar.set_description(f"Epoch {epoch}")
    
    for it, batch in pbar:
        # YOUR CODE GOES HERE
        with torch.no_grad():
            text, text_lengths = batch.text
            y_true = batch.label

            y_pred = model(text, text_lengths.cpu())

            loss = loss_func(y_pred[:, 1], y_true)

            val_loss += loss.cpu().item()
            val_f1_score += f1_score(y_true.cpu().numpy(), torch.argmax(y_pred, dim=1).cpu().numpy())
    val_loss /= len(torchtext_valid_dataloader)
    val_f1_score /= len(torchtext_valid_dataloader)

    if val_loss < min_loss:
        min_loss = val_loss
        best_model = model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
    print("---------  Training f1_score: {}, Validation f1_score: {}".format(train_f1_score, val_f1_score))

model.load_state_dict(best_model)

  0%|          | 0/274 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.6295029827713097, Validation Loss: 0.5650419489306918
---------  Training f1_score: 0.6102908851470303, Validation f1_score: 0.7394398467345911


  0%|          | 0/274 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

...

  0%|          | 0/118 [00:00<?, ?it/s]

Epoch: 9, Training Loss: 0.03448475367165711, Validation Loss: 0.7150087938722918
---------  Training f1_score: 0.9881785788124593, Validation f1_score: 0.8525301866601632


  0%|          | 0/274 [00:00<?, ?it/s]

  0%|          | 0/118 [00:00<?, ?it/s]

<All keys matched successfully>

In [ ]:
test_loss = 0.0
test_f1_score = 0.0

model.eval()

pbar = tqdm(enumerate(torchtext_test_dataloader), total=len(torchtext_test_dataloader), leave=False)

for it, batch in pbar:
    with torch.no_grad():
        text, text_lengths = batch.text
        y_true = batch.label

        y_pred = model(text, text_lengths.cpu())

        loss = loss_func(y_pred[:, 1], y_true)

        test_loss += loss.cpu().item()
        test_f1_score += f1_score(y_true.cpu().numpy(), torch.argmax(y_pred, dim=1).cpu().numpy())
test_loss /= len(torchtext_test_dataloader)
test_f1_score /= len(torchtext_test_dataloader)

print("Testing Loss: {}".format(test_loss))
print("Testing f1_score: {}".format(test_f1_score))

  0%|          | 0/391 [00:00<?, ?it/s]

Testing Loss: 0.9582511080652857
Testing f1_score: 0.8278642686492178


Посчитайте f1-score вашего классификатора на тестовом датасете.

**Ответ**:

In [ ]:
# Testing f1_score: 0.8278642686492178

## CNN

![](https://www.researchgate.net/publication/333752473/figure/fig1/AS:769346934673412@1560438011375/Standard-CNN-on-text-classification.png)

Для классификации текстов также часто используют сверточные нейронные сети. Идея в том, что как правило сентимент содержат словосочетания из двух-трех слов, например "очень хороший фильм" или "невероятная скука". Проходясь сверткой по этим словам мы получим какой-то большой скор и выхватим его с помощью MaxPool. Далее идет обычная полносвязная сетка. Важный момент: свертки применяются не последовательно, а параллельно. Давайте попробуем!

In [6]:
TEXT = Field(sequential=True, lower=True, batch_first=True)  # batch_first тк мы используем conv  
LABEL = LabelField(batch_first=True, dtype=torch.float)

train_Dataset1, test_Dataset1 = datasets.IMDB.splits(TEXT, LABEL)
train_Dataset1, valid_Dataset1 = train_Dataset1.split(random_state=random.seed(SEED))

TEXT.build_vocab(train_Dataset1)
LABEL.build_vocab(train_Dataset1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 36.6MB/s]


In [7]:
len(train_Dataset1), len(valid_Dataset1), len(test_Dataset1)

(17500, 7500, 25000)

In [8]:
torchtext_train_dataloader1, torchtext_valid_dataloader1, torchtext_test_dataloader1 = BucketIterator.splits(
        (train_Dataset1, valid_Dataset1, test_Dataset1),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.text),
        sort_within_batch=True,
        device=device,
        repeat=False,
)

In [9]:
len(torchtext_train_dataloader1), len(torchtext_valid_dataloader1), len(torchtext_test_dataloader1)

(137, 30, 98)

In [10]:
torchtext_train_dataloader1.create_batches()
torchtext_valid_dataloader1.create_batches()
torchtext_test_dataloader1.create_batches()

Вы можете использовать Conv2d с `in_channels=1, kernel_size=(kernel_sizes[0], emb_dim))` или Conv1d c `in_channels=emb_dim, kernel_size=kernel_size[0]`. Но хорошенько подумайте над shape в обоих случаях.

In [11]:
emb_dim = 100
kernel_sizes = [3, 4, 5]
out_channels=64

In [12]:
# достанем один батч, чтобы мы могл отслеживать размеры тензора, проходящего
# через модель

In [13]:
for batch_no, batch in enumerate(torchtext_train_dataloader1):
    text = batch.text # text.size() -> seq_len, batch_size
    label = batch.label
    print(text)
    print(label)
    break

tensor([[   49,    88,  4630,  ..., 19563,   116,  4842],
        [    2,  1282,    17,  ..., 10491,     1,     1],
        [   10,     7,     3,  ...,     1,     1,     1],
        ...,
        [10545, 10545,    10,  ...,     1,     1,     1],
        [    9,  4649,   309,  ...,     1,     1,     1],
        [    2, 10868,     7,  ...,     1,     1,     1]], device='cuda:0')
tensor([1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0.,
        1., 0., 1., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
        0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1.,
        0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1.,
        0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1.,
        1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1.,
        1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
        1., 1.], device='cuda:0')


In [29]:
text = text.cpu()
label = label.cpu()

In [31]:
text.size(), label.size()

(torch.Size([168, 2278]), torch.Size([128]))

In [32]:
# Небольшое изменение в процессе корректировки
# Т.к. последующий шаг по интерпретации модели не отрабатаывает, сетую на то, что это
# связано с тем, что модель возвращает 2 класса, а не 1, как подразумевалось авторами,
# поэтому я видоизменяю выход модели, и слегка передылваю train_val_loop, по сравнению
# с реализацией аналогичных пунктов, приведенных выше

In [33]:
# Будем итеративно строить нашу модель 
model_ = nn.Sequential()
model_.add_module("emd", nn.Embedding(len(TEXT.vocab), emb_dim))

In [34]:
b1 = model_(text)
b1.size() # batch_size, seq_length, embedding_dim

torch.Size([168, 2278, 100])

In [35]:
# поменяем порядок , чтобы мы правильно применяли свёрточные слои
class Permute(nn.Module):

    def forward(self, x):
        return  x.permute((0, 2, 1))

In [36]:
model_.add_module("permute", Permute())

In [37]:
b2 = model_(text)
b2.size() # batch_size, embedding_dim, seq_length

torch.Size([168, 100, 2278])

In [38]:
# Теперь, когда резмерности батча приведены в правильный порядок добавим свёртки

model_.add_module("conv1", nn.Conv1d(in_channels=emb_dim, out_channels=out_channels, kernel_size=kernel_sizes[1]))

In [39]:
b3 = model_(text)
b3.size()

torch.Size([168, 64, 2275])

In [40]:
# также необходимо добавить пулинги

model_.add_module("pool1", nn.MaxPool1d(kernel_size=b3.size(2)))

In [41]:
b4 = model_(text)
b4.size()

torch.Size([168, 64, 1])

In [42]:
model_ # пример структуры получившейся модели, теперь реализуем полноценную модельку на основе экспериментов выше

Sequential(
  (emd): Embedding(201383, 100)
  (permute): Permute()
  (conv1): Conv1d(100, 64, kernel_size=(4,), stride=(1,))
  (pool1): MaxPool1d(kernel_size=2275, stride=2275, padding=0, dilation=1, ceil_mode=False)
)

In [15]:
kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT.vocab)
emb_dim = 100
hidden_dim = 256
out_channels = 64
out_channel = 1
dropout = 0.5

In [16]:
class CNN(nn.Module):
    def __init__(
        self,
        vocab_size,
        emb_dim,
        out_channels,
        kernel_sizes,
        dropout=0.5,
        out_channel=1
    ):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, emb_dim)

        # YOUR CODE GOES HERE
        self.conv_0 = nn.Conv1d(in_channels=emb_dim, out_channels=out_channels, kernel_size=kernel_sizes[0])  
        self.conv_1 = nn.Conv1d(in_channels=emb_dim, out_channels=out_channels, kernel_size=kernel_sizes[0])
        self.conv_2 = nn.Conv1d(in_channels=emb_dim, out_channels=out_channels, kernel_size=kernel_sizes[0])
                                
        self.dropout = nn.Dropout(dropout)

        self.fc = nn.Linear(len(kernel_sizes) * out_channels, out_channel)
        
        
    def forward(self, text):
        
        embedded = self.embedding(text) # batch_size, seq_length, emb_dim
        
        embedded = embedded.permute((0, 2, 1)) # batch_size, emb_dim, seq_length
        
        conved_0 = F.relu(self.conv_0(embedded)) # batch_size, out_channels, *
        conved_1 = F.relu(self.conv_1(embedded))  # may be reshape here
        conved_2 = F.relu(self.conv_2(embedded))  # may be reshape here
        
        pooled_0 = F.max_pool1d(conved_0, conved_0.shape[2]).squeeze(2)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))
            
        return self.fc(cat)

In [17]:
cnn_model = CNN(vocab_size=vocab_size, emb_dim=emb_dim, out_channels=out_channels,
            kernel_sizes=kernel_sizes, dropout=dropout)

In [21]:
cnn_model.to(device)

CNN(
  (embedding): Embedding(201383, 100)
  (conv_0): Conv1d(100, 64, kernel_size=(3,), stride=(1,))
  (conv_1): Conv1d(100, 64, kernel_size=(3,), stride=(1,))
  (conv_2): Conv1d(100, 64, kernel_size=(3,), stride=(1,))
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=192, out_features=1, bias=True)
)

In [22]:
opt = torch.optim.Adam(cnn_model.parameters())
loss_func = nn.BCEWithLogitsLoss()

In [23]:
max_epochs = 20
patience = 7

Обучите!

In [27]:
import numpy as np

min_loss = np.inf

cur_patience = 0

for epoch in range(1, max_epochs + 1):

    train_f1_score = 0.0
    train_loss = 0.0

    cnn_model.train()

    pbar = tqdm(enumerate(torchtext_train_dataloader1), total=len(torchtext_train_dataloader1), leave=False)
    pbar.set_description(f"Epoch {epoch}")

    for it, batch in pbar: 
        #YOUR CODE GOES HERE
        text = batch.text
        y_true = batch.label
        
        opt.zero_grad()
        
        y_pred = cnn_model(text) # (batch_size, 1)

        loss = loss_func(y_pred, y_true)
        loss.backward()
        opt.step()

        train_loss += loss.detach().cpu().item()
        train_f1_score += f1_score(y_true.cpu().numpy(), (torch.sigmoid(y_pred).cpu() > 0.5).float().numpy())
    train_loss /= len(torchtext_train_dataloader1)
    train_f1_score /= len(torchtext_train_dataloader1)

    val_f1_score = 0.0
    val_loss = 0.0

    cnn_model.eval()

    pbar = tqdm(enumerate(torchtext_valid_dataloader1), total=len(torchtext_valid_dataloader1), leave=False)
    pbar.set_description(f"Epoch {epoch}")

    for it, batch in pbar:
        # YOUR CODE GOES HERE
        with torch.no_grad():
            text = batch.text
            y_true = batch.label

            y_pred = cnn_model(text).squeeze()

            loss = loss_func(y_pred, y_true)

            val_loss += loss.cpu().item()
            val_f1_score += f1_score(y_true.cpu().numpy(), (torch.sigmoid(y_pred).cpu() > 0.5).float().numpy())
    val_loss /= len(torchtext_valid_dataloader1)
    val_f1_score /= len(torchtext_valid_dataloader1)

    if val_loss < min_loss:
        min_loss = val_loss
        best_model = cnn_model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
    print("---------  Training f1_score: {}, Validation f1_score: {}".format(train_f1_score, val_f1_score))

cnn_model.load_state_dict(best_model)

  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.48682590884013766, Validation Loss: 0.43820490340391793
---------  Training f1_score: 0.7641495323186296, Validation f1_score: 0.8080211969677539


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 13, Training Loss: 0.06441939048414683, Validation Loss: 0.45489110549290973
---------  Training f1_score: 0.9784241308437239, Validation f1_score: 0.8395926109837831


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

<All keys matched successfully>

In [28]:
test_loss = 0.0
test_f1_score = 0.0

cnn_model.eval()

pbar = tqdm(enumerate(torchtext_test_dataloader1), total=len(torchtext_test_dataloader1), leave=False)

for it, batch in pbar:
    with torch.no_grad():
        text = batch.text
        y_true = batch.label

        y_pred = cnn_model(text).squeeze()

        loss = loss_func(y_pred, y_true)

        test_loss += loss.cpu().item()
        test_f1_score += f1_score(y_true.cpu().numpy(), (torch.sigmoid(y_pred).cpu() > 0.5).float().numpy())
test_loss /= len(torchtext_test_dataloader1)
test_f1_score /= len(torchtext_test_dataloader1)

print("Testing Loss: {}".format(test_loss))
print("Testing f1_score: {}".format(test_f1_score))

  0%|          | 0/98 [00:00<?, ?it/s]

Testing Loss: 0.5149682751115487
Testing f1_score: 0.8255127660813958


Посчитайте f1-score вашего классификатора.

**Ответ**:

In [ ]:
# Testing f1_score: 0.8255127660813958

## Интерпретируемость

Посмотрим, куда смотрит наша модель. Достаточно запустить код ниже.

In [7]:
!pip install -q captum

     |████████████████████████████████| 1.4 MB 4.2 MB/s 


In [18]:
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

PAD_IND = TEXT.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(cnn_model, cnn_model.embedding)

In [19]:
def forward_with_softmax(inp):
    logits = model(inp)
    return torch.softmax(logits, 0)[0][1]

def forward_with_sigmoid(input):
    return torch.sigmoid(cnn_model(input))


# accumalate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 7, label = 0):
    model.eval()
    text = [tok for tok in TEXT.tokenize(sentence)]
    if len(text) < min_len:
        text += ['pad'] * (min_len - len(text))
    indexed = [TEXT.vocab.stoi[t] for t in text]

    model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    # input_indices dim: [sequence_length]
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid(input_indices).squeeze()

    pred_ind = round(pred.item())

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=5000, return_convergence_delta=True)

    print('pred: ', LABEL.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            LABEL.vocab.itos[pred_ind],
                            LABEL.vocab.itos[label],
                            LABEL.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))

In [60]:
interpret_sentence(cnn_model, 'It was a fantastic performance !', label=1)
interpret_sentence(cnn_model, 'Best film ever', label=1)
interpret_sentence(cnn_model, 'Such a great show!', label=1)
interpret_sentence(cnn_model, 'It was a horrible movie', label=0)
interpret_sentence(cnn_model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(cnn_model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.96 ) , delta:  tensor([8.9544e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.00 ) , delta:  tensor([3.2459e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.11 ) , delta:  tensor([1.1704e-06], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.00 ) , delta:  tensor([1.1465e-06], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.08 ) , delta:  tensor([4.0419e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.38 ) , delta:  tensor([6.6384e-05], device='cuda:0', dtype=torch.float64)


Попробуйте добавить свои примеры!

In [63]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.96),pos,1.39,It was a fantastic performance ! pad
pos,neg (0.00),pos,1.57,Best film ever pad pad pad pad
pos,neg (0.11),pos,1.26,Such a great show! pad pad pad
neg,neg (0.00),pos,-0.18,It was a horrible movie pad pad
neg,neg (0.08),pos,0.92,I've never watched something as bad pad
neg,neg (0.38),pos,1.24,It is a disgusting movie! pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.96),pos,1.39,It was a fantastic performance ! pad
pos,neg (0.00),pos,1.57,Best film ever pad pad pad pad
pos,neg (0.11),pos,1.26,Such a great show! pad pad pad
neg,neg (0.00),pos,-0.18,It was a horrible movie pad pad
neg,neg (0.08),pos,0.92,I've never watched something as bad pad
neg,neg (0.38),pos,1.24,It is a disgusting movie! pad pad


## Эмбеддинги слов

Вы ведь не забыли, как мы можем применить знания о word2vec и GloVe. Давайте попробуем!

In [20]:
TEXT2 = Field(sequential=True, use_vocab=True, lower=True, batch_first=True)    
LABEL2 = LabelField(use_vocab=True, batch_first=True, dtype=torch.float)

In [21]:
train_Dataset2, test_Dataset2 = datasets.IMDB.splits(TEXT2, LABEL2)  # загрузим датасет
train_Dataset2, valid_Dataset2 = train_Dataset2.split(random_state=random.seed(SEED))

In [22]:
TEXT2.build_vocab(train_Dataset2, vectors="glove.42B.300d")
LABEL2.build_vocab(train_Dataset2)

.vector_cache/glove.42B.300d.zip: 1.88GB [06:40, 4.68MB/s]                            
100%|█████████▉| 1917493/1917494 [04:18<00:00, 7420.99it/s]


In [23]:
# посмотрим, что получилось
TEXT2.vocab.stoi

defaultdict(<bound method Vocab._default_unk_index of <torchtext.legacy.vocab.Vocab object at 0x7feb6b81b3d0>>,
            {'<unk>': 0,
             '<pad>': 1,
             'the': 2,
             'a': 3,
             'and': 4,
             'of': 5,
             'to': 6,
             'is': 7,
             'in': 8,
             'i': 9,
             'this': 10,
             'that': 11,
             'it': 12,
             '/><br': 13,
             'was': 14,
             'as': 15,
             'for': 16,
             'with': 17,
             'but': 18,
             'on': 19,
             'movie': 20,
             'his': 21,
             'not': 22,
             'are': 23,
             'film': 24,
             'you': 25,
             'have': 26,
             'he': 27,
             'be': 28,
             'one': 29,
             'at': 30,
             'by': 31,
             'an': 32,
             'they': 33,
             'from': 34,
             'all': 35,
             'who': 36,
           

In [24]:
TEXT2.vocab.vectors.size()

torch.Size([201383, 300])

In [25]:
TEXT2.vocab.freqs

Counter({'she': 8792,
         'may': 2290,
         'have': 19147,
         'an': 14857,
         'oscar': 419,
         'and': 110723,
         'a': 112087,
         'golden': 163,
         'globe,': 6,
         'but': 27891,
         'this': 48426,
         'film': 19330,
         'shows': 1355,
         'why': 3058,
         'also': 5634,
         'is': 72992,
         'perennial': 14,
         'razzie': 14,
         'nominee.': 3,
         'to': 93491,
         'do': 5486,
         'that': 46359,
         'so': 12699,
         'bad': 4820,
         'must': 2076,
         'be': 17927,
         'indication': 46,
         'needs': 557,
         'money.': 200,
         'could': 5305,
         'ads': 24,
         'on': 22099,
         'you': 19271,
         "shouldn't": 225,
         'talk': 501,
         'cell': 74,
         'phone': 165,
         'while': 3301,
         'driving,': 8,
         'especially': 1511,
         'at': 15766,
         'night': 999,
         'the': 225243,
  

In [26]:
LABEL2.vocab.freqs

Counter({'neg': 8810, 'pos': 8690})

In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [28]:
torchtext_train_dataloader2, torchtext_valid_dataloader2, torchtext_test_dataloader2 = BucketIterator.splits(
        (train_Dataset2, valid_Dataset2, test_Dataset2),
        batch_sizes=(128, 256, 256),
        sort=False,
        sort_key= lambda x: len(x.text),
        sort_within_batch=True,
        device=device,
        repeat=False,
)

In [29]:
torchtext_train_dataloader2.create_batches()
torchtext_valid_dataloader2.create_batches()
torchtext_test_dataloader2.create_batches()

In [36]:
word_embeddings = TEXT2.vocab.vectors

kernel_sizes = [3, 4, 5]
vocab_size = len(TEXT2.vocab)
dropout = 0.5
dim = 300

In [37]:
cnn_model = CNN(vocab_size=vocab_size, emb_dim=dim, out_channels=64,
            kernel_sizes=kernel_sizes, dropout=dropout, out_channel=1)

In [38]:
cnn_model.embedding.weight = nn.Parameter(word_embeddings)

In [39]:
cnn_model.to(device)

opt = torch.optim.Adam(cnn_model.parameters())
loss_func = nn.BCEWithLogitsLoss()

Вы знаете, что делать.

In [40]:
import numpy as np

min_loss = np.inf

cur_patience = 0
max_epochs = 30
patience = 10

for epoch in range(1, max_epochs + 1):

    train_f1_score = 0.0
    train_loss = 0.0

    cnn_model.train()

    pbar = tqdm(enumerate(torchtext_train_dataloader2), total=len(torchtext_train_dataloader2), leave=False)
    pbar.set_description(f"Epoch {epoch}")

    for it, batch in pbar: 
        #YOUR CODE GOES HERE
        text = batch.text
        y_true = batch.label
        
        opt.zero_grad()
        
        y_pred = cnn_model(text).squeeze() # (batch_size, 1)

        loss = loss_func(y_pred, y_true)
        loss.backward()
        opt.step()

        train_loss += loss.detach().cpu().item()
        train_f1_score += f1_score(y_true.cpu().numpy(), (torch.sigmoid(y_pred).cpu() > 0.5).float().numpy())
    train_loss /= len(torchtext_train_dataloader2)
    train_f1_score /= len(torchtext_train_dataloader2)

    val_f1_score = 0.0
    val_loss = 0.0

    cnn_model.eval()

    pbar = tqdm(enumerate(torchtext_valid_dataloader2), total=len(torchtext_valid_dataloader2), leave=False)
    pbar.set_description(f"Epoch {epoch}")

    for it, batch in pbar:
        # YOUR CODE GOES HERE
        with torch.no_grad():
            text = batch.text
            y_true = batch.label

            y_pred = cnn_model(text).squeeze()

            loss = loss_func(y_pred, y_true)

            val_loss += loss.cpu().item()
            val_f1_score += f1_score(y_true.cpu().numpy(), (torch.sigmoid(y_pred).cpu() > 0.5).float().numpy())
    val_loss /= len(torchtext_valid_dataloader2)
    val_f1_score /= len(torchtext_valid_dataloader2)

    if val_loss < min_loss:
        min_loss = val_loss
        best_model = cnn_model.state_dict()
    else:
        cur_patience += 1
        if cur_patience == patience:
            cur_patience = 0
            break
    
    print('Epoch: {}, Training Loss: {}, Validation Loss: {}'.format(epoch, train_loss, val_loss))
    print("---------  Training f1_score: {}, Validation f1_score: {}".format(train_f1_score, val_f1_score))

cnn_model.load_state_dict(best_model)

  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 1, Training Loss: 0.5170622289615826, Validation Loss: 0.37479890485604606
---------  Training f1_score: 0.7227050853277115, Validation f1_score: 0.8484763487973538


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: 12, Training Loss: 0.0018969821560121801, Validation Loss: 0.4754878282546997
---------  Training f1_score: 0.9998961298803763, Validation f1_score: 0.8699650309897676


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

<All keys matched successfully>

In [42]:
test_loss = 0.0
test_f1_score = 0.0

cnn_model.eval()

pbar = tqdm(enumerate(torchtext_test_dataloader2), total=len(torchtext_test_dataloader2), leave=False)

for it, batch in pbar:
    with torch.no_grad():
        text = batch.text
        y_true = batch.label

        y_pred = cnn_model(text).squeeze()

        loss = loss_func(y_pred, y_true)

        test_loss += loss.cpu().item()
        test_f1_score += f1_score(y_true.cpu().numpy(), (torch.sigmoid(y_pred).cpu() > 0.5).float().numpy())
test_loss /= len(torchtext_test_dataloader2)
test_f1_score /= len(torchtext_test_dataloader2)

print("Testing Loss: {}".format(test_loss))
print("Testing f1_score: {}".format(test_f1_score))

  0%|          | 0/98 [00:00<?, ?it/s]

Testing Loss: 0.47554718201257745
Testing f1_score: 0.8591374029459167


Посчитайте f1-score вашего классификатора.

**Ответ**:

In [43]:
# Testing f1_score: 0.8591374029459167

# как можно заметить инициализация предобученными эмбеддингами улучшает качество предсказания модели

Проверим насколько все хорошо!

In [45]:
PAD_IND = TEXT2.vocab.stoi['pad']

token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(cnn_model, cnn_model.embedding)
vis_data_records_ig = []

interpret_sentence(cnn_model, 'It was a fantastic performance !', label=1)
interpret_sentence(cnn_model, 'Best film ever', label=1)
interpret_sentence(cnn_model, 'Such a great show!', label=1)
interpret_sentence(cnn_model, 'It was a horrible movie', label=0)
interpret_sentence(cnn_model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(cnn_model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.98 ) , delta:  tensor([0.0003], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.00 ) , delta:  tensor([3.9156e-05], device='cuda:0', dtype=torch.float64)
pred:  pos ( 0.78 ) , delta:  tensor([4.6364e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.00 ) , delta:  tensor([6.9329e-06], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.22 ) , delta:  tensor([7.0864e-05], device='cuda:0', dtype=torch.float64)
pred:  neg ( 0.00 ) , delta:  tensor([7.4362e-05], device='cuda:0', dtype=torch.float64)


In [46]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.98),pos,1.66,It was a fantastic performance ! pad
pos,neg (0.00),pos,1.32,Best film ever pad pad pad pad
pos,pos (0.78),pos,1.44,Such a great show! pad pad pad
neg,neg (0.00),pos,-0.19,It was a horrible movie pad pad
neg,neg (0.22),pos,1.44,I've never watched something as bad pad
neg,neg (0.00),pos,-0.29,It is a disgusting movie! pad pad


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.98),pos,1.66,It was a fantastic performance ! pad
pos,neg (0.00),pos,1.32,Best film ever pad pad pad pad
pos,pos (0.78),pos,1.44,Such a great show! pad pad pad
neg,neg (0.00),pos,-0.19,It was a horrible movie pad pad
neg,neg (0.22),pos,1.44,I've never watched something as bad pad
neg,neg (0.00),pos,-0.29,It is a disgusting movie! pad pad
